# Анализ тональности текста (tweet'а)

Цель: построить классификатор для задачи анализа тональности текстов на основе сверточных нейронных сетей и LSTM модели.

Данные: сообщения из Twitter за 2016 год.

## Загрузка данных

In [1]:
import pandas as pd

df = pd.read_csv('./data/data.csv', header=None, usecols=[1, 2], names=['label', 'text'], sep='\t')

In [2]:
df.head()

,label,text
0,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,neutral,Order Go Set a Watchman in store or through ou...
2,negative,If these runway renovations at the airport pre...
3,neutral,If you could ask an onstage interview question...
4,positive,A portion of book sales from our Harper Lee/Go...


In [3]:
df['label'].value_counts().plot(kind='bar');

## Подготовка данных

In [4]:
MAX_TEXT_LENGTH = 140

# удаление слишком длинных записей
df = df[df['text'].map(len) <= MAX_TEXT_LENGTH]

In [5]:
# объединение нейтральных и негативных записей
df['label'] = df['label'].replace('neutral', 'negative')

# количество записей для каждой метки
NUMBER_PER_LABEL = min(df['label'].value_counts())

# семплирование на равные части
df = df.groupby('label').apply(lambda x: x.sample(NUMBER_PER_LABEL))

In [6]:
# one-hot encoding
df = df.join(pd.get_dummies(df['label']))

In [7]:
# замена ссылок на токен "URL"
df['text'] = df['text'].str.replace(r'https?://\S+', 'URL')

# замена упоминаний пользователя на токен "USER"
df['text'] = df['text'].str.replace(r'@[^\s]+', 'USER')

# удаление пунктуации
df['text'] = df['text'].str.replace(r'[^a-zA-Z0-9]+', ' ')

# приведение к нижнему регистру
df['text'] = df['text'].str.lower()

In [8]:
# токенизация
import nltk

df['tokens'] = df['text'].apply(nltk.word_tokenize)

In [9]:
# лемматизация
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmatize = lambda tokens: [lemmatizer.lemmatize(token) for token in tokens]
df['tokens'] = df['tokens'].apply(lemmatize)

In [10]:
# удаление стоп-слов
from nltk.corpus import stopwords

STOP_WORDS = set(stopwords.words('english'))

delete_stop_words = lambda words: [word for word in words if word not in STOP_WORDS]
df['tokens'] = df['tokens'].apply(delete_stop_words)

In [11]:
# удаление редких слов
from collections import Counter

MINIMAL_WORD_FREQUENCY = 3

counter = Counter()
for row in df['tokens']:
    counter.update(row)

delete_rare_words = lambda words: [word for word in words if counter[word] >= MINIMAL_WORD_FREQUENCY]
df['tokens'] = df['tokens'].apply(delete_rare_words)

### Разбиение выборки

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.1)

y_train, y_test = df_train[['negative', 'positive']].values, df_test[['negative', 'positive']].values

### Векторизация текстов

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# максимальное количество слов в тексте 
MAX_SEQUENCE_LENGTH = df['tokens'].map(len).max()

# количество слов в словаре
NUM_WORDS = 5000

def get_sequences(tokenizer, texts):
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# создание и обучение токенизатора
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(df_train['tokens'].tolist())

# преобразование текстов в массивы идентификаторов токенов
x_train = get_sequences(tokenizer, df_train['tokens'].values)
x_test = get_sequences(tokenizer, df_test['tokens'].values)

Using TensorFlow backend.


### Векторизация слов

In [14]:
import multiprocessing

import numpy as np

import gensim
from gensim.models import Word2Vec

# размерность вектора слова
EMBEDDING_DIM = 200

# создание и обучение модели Word2Vec 
word2vec = Word2Vec(df['tokens'].values, size=EMBEDDING_DIM, window=5, min_count=3, workers=multiprocessing.cpu_count())

# построение матрицы весов embedding-слоя
embedding_matrix = np.zeros((NUM_WORDS, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    if i == NUM_WORDS:
        break
    embedding_matrix[i] = word2vec.wv[word]

## Построение классификаторов

### Объявление метрик

In [15]:
from keras import backend as K

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

metrics = [precision, recall, f1]

### LSTM

In [16]:
from keras.layers import Input, Dense, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.models import Sequential

def build_lstm(max_sequence_length, num_words, embedding_dim, embedding_matrix, labels_index):
    model = Sequential()
    model.add(Embedding(num_words, embedding_dim, input_length=max_sequence_length))
    model.add(LSTM(embedding_dim))
    model.add(Dense(labels_index, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
    return model

In [17]:
lstm = build_lstm(MAX_SEQUENCE_LENGTH, NUM_WORDS, EMBEDDING_DIM, embedding_matrix, 2) 
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 26, 200)           1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 1,321,202
Trainable params: 1,321,202
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = lstm.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.1)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 15846 samples, validate on 1761 samples
Epoch 1/5
15846/15846 [==============================] - 67s 4ms/step - loss: 0.5731 - precision: 0.7005 - recall: 0.7010 - f1: 0.7006 - val_loss: 0.5334 - val_precision: 0.7162 - val_recall: 0.7154 - val_f1: 0.7158
Epoch 2/5
15846/15846 [==============================] - 62s 4ms/step - loss: 0.4570 - precision: 0.7850 - recall: 0.7845 - f1: 0.7847 - val_loss: 0.5346 - val_precision: 0.7196 - val_recall: 0.7204 - val_f1: 0.7200
Epoch 3/5
15846/15846 [==============================] - 66s 4ms/step - loss: 0.3809 - precision: 0.8241 - recall: 0.8243 - f1: 0.8242 - val_loss: 0.5861 - val_precision: 0.7111 - val_recall: 0.7098 - val_f1: 0.7105
Epoch 4/5
15846/15846 [==============================] - 50s 3ms/step - loss: 0.3083 - precision: 0.8637 - recall: 0.8635 - f1: 0.8636 - val_loss: 0.7023 - val_precision: 0.7148 - val_recall: 0.7143 - val_f1: 0.7145
Epoch 5/5
15846/15846 [==============================] - 55s 3ms/step - loss: 0.2491 - 

In [19]:
from sklearn.metrics import classification_report

predicted = np.round(lstm.predict(x_test))
lstm_metrics = classification_report(y_test, predicted, digits=4,  zero_division=0)
print(lstm_metrics)

              precision    recall  f1-score   support

           0     0.6877    0.6949    0.6913       957
           1     0.7040    0.6970    0.7005      1000

   micro avg     0.6960    0.6960    0.6960      1957
   macro avg     0.6959    0.6959    0.6959      1957
weighted avg     0.6960    0.6960    0.6960      1957
 samples avg     0.6955    0.6960    0.6956      1957



### CNN

In [20]:
from keras.layers import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Model

def build_cnn(max_sequence_length, num_words, embedding_dim, embedding_matrix, labels_index):
    input_layer = Input(shape=(max_sequence_length,), dtype='int32')
    encoder_layer = Embedding(num_words, embedding_dim, input_length=max_sequence_length)(input_layer)
    
    filter_layers = []
    
    for filter_size in range(2, 7):
        layer = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(encoder_layer)
        layer = GlobalMaxPooling1D()(layer)
        filter_layers.append(layer)
    
    x = concatenate(filter_layers, axis=1)
    x = Dropout(0.1)(x)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    output_layer = Dense(labels_index, activation='sigmoid')(x)
    
    model = Model(input_layer, output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
    return model

In [21]:
cnn = build_cnn(MAX_SEQUENCE_LENGTH, NUM_WORDS, EMBEDDING_DIM, embedding_matrix, 2) 
cnn.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 26, 200)      1000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 200)      80200       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 24, 200)      120200      embedding_2[0][0]                
____________________________________________________________________________________________

In [22]:
history = cnn.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.1)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 15846 samples, validate on 1761 samples
Epoch 1/5
15846/15846 [==============================] - 60s 4ms/step - loss: 0.5742 - precision: 0.6907 - recall: 0.6906 - f1: 0.6902 - val_loss: 0.5328 - val_precision: 0.7083 - val_recall: 0.7160 - val_f1: 0.7121
Epoch 2/5
15846/15846 [==============================] - 41s 3ms/step - loss: 0.4363 - precision: 0.7969 - recall: 0.7968 - f1: 0.7967 - val_loss: 0.5504 - val_precision: 0.7184 - val_recall: 0.7210 - val_f1: 0.7197
Epoch 3/5
15846/15846 [==============================] - 31s 2ms/step - loss: 0.2780 - precision: 0.8807 - recall: 0.8816 - f1: 0.8811 - val_loss: 0.6393 - val_precision: 0.7013 - val_recall: 0.6987 - val_f1: 0.6999
Epoch 4/5
15846/15846 [==============================] - 33s 2ms/step - loss: 0.1131 - precision: 0.9573 - recall: 0.9575 - f1: 0.9574 - val_loss: 1.1494 - val_precision: 0.6961 - val_recall: 0.6970 - val_f1: 0.6965
Epoch 5/5
15846/15846 [==============================] - 30s 2ms/step - loss: 0.0597 - 

In [23]:
predicted = np.round(cnn.predict(x_test))
cnn_metrics = classification_report(y_test, predicted, digits=4,  zero_division=0)
print(cnn_metrics)

              precision    recall  f1-score   support

           0     0.6581    0.7220    0.6886       957
           1     0.7059    0.6480    0.6757      1000

   micro avg     0.6804    0.6842    0.6823      1957
   macro avg     0.6820    0.6850    0.6821      1957
weighted avg     0.6825    0.6842    0.6820      1957
 samples avg     0.6814    0.6842    0.6823      1957



In [24]:
output = 'output.txt'
with open(output, 'a') as f:
    f.write('2016, 2 класса, LSTM')
    f.write('\n')        
    f.write(lstm_metrics)
    f.write('\n')        
    f.write('2016, 2 класса, CNN')
    f.write('\n')        
    f.write(cnn_metrics)
    f.write('\n')